### CVPR2023 UG2 + Challenge - Object Detection in Haze

In [2]:
# setup path
%cd "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib"

import os
import sys

os.environ["PYTHONPATH"]="/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib"
sys.path.append('/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/ultralytics')

/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib


In [3]:
# main code to run YOLO training
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data="/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/Hazy.yaml", epochs=150, pretrained=True, iou=0.9, visualize=True, patience=0)  # train the model
results = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.17 🚀 Python-3.9.16 torch-2.0.0+cu117 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/Hazy.yaml, epochs=150, patience=0, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.9, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=True, augment=False, agnostic_nms=False, retina_masks=False, classes=None, format=torchscript, keras=False, optimize=False, int8=

In [ ]:
#  inference on testing dataset
!chmod 755 bin/yolo
PATH="runs/detect/train59 predict6 hazy iou0.9 150 epochs/"
!bin/yolo task=detect mode=predict model='{PATH}/weights/best.pt' save_txt=True conf=0.25 save_conf=True source='/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/test'

#### Helper Functions

In [ ]:
# convert Pascal VOC format into Yolo
from numpy.lib.type_check import imag
import re
import os
import cv2

img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/train/haze_images"
label_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/train/haze_images_labels"
dest_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/labels/train"

for filename_jpg in os.listdir(img_path):
  yolo = ""
  img = cv2.imread(os.path.join(img_path, filename_jpg))
  img_H = img.shape[0]
  img_W = img.shape[1]
  filename_txt = os.path.join(label_path, filename_jpg.replace("jpg", "txt"))
  if filename_txt.endswith(".txt"):
    with open(os.path.join(label_path, filename_txt), 'r') as f:
      lines = f.readlines()
      for line in lines:
        x_min, y_min, x_max, y_max = [int(bbox) for bbox in line.split()[1:]]
        x_center = round((x_max + x_min) / 2  / img_W, 4)
        y_center = round((y_max + y_min) / 2 / img_H, 4)
        width = round((x_max - x_min) / img_W, 4)
        height = round((y_max - y_min) / img_H, 4)
        yolo += (f"0 {x_center} {y_center} {width} {height}\n")
    print(yolo)
    print(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")))
    with open(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")), 'w') as f:
      f.write(yolo)


In [ ]:
# convert YOLO format into Pascal VOC
from numpy.lib.type_check import imag
import re
import os
import cv2

img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/test"
label_path = "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/runs/detect/predict4/labels"
dest_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results"

for filename_jpg in os.listdir(img_path):
  pascal_voc = ""
  img = cv2.imread(os.path.join(img_path, filename_jpg))
  img_H = img.shape[0]
  img_W = img.shape[1]
  filename_txt = os.path.join(label_path, filename_jpg.replace("jpg", "txt"))
  try:
    if filename_txt.endswith(".txt"):
      with open(os.path.join(label_path, filename_txt), 'r') as f:
        lines = f.readlines()
        for line in lines:
          x_center, y_center, width, height, confid = [float(bbox) for bbox in line.split()[1:]]
          x_min = (x_center - width/2) * img_W
          y_min = (y_center - height/2) * img_H
          x_max = (x_center + width/2) * img_W
          y_max = (y_center + height/2) * img_H
          pascal_voc += (f"vehicle {x_min:.2f} {y_min:.2f} {x_max:.2f} {y_max:.2f} {confid:.2f}\n")
          print(pascal_voc)
          print(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")))
      with open(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")), 'w') as f:
        f.write(pascal_voc)
  except:
    pass

In [5]:
# convert YOLO format into UC2 tmp_class_name, left, top, right, bottom, confidence format
from numpy.lib.type_check import imag
import re
import os
import cv2

img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/test"
label_path = "/content/drive/MyDrive/Colab Notebooks/Bruce/Yolo8-lib/runs/detect/predict4/labels"
dest_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results"

for filename_jpg in os.listdir(img_path):
  pascal_voc = ""
  img = cv2.imread(os.path.join(img_path, filename_jpg))
  img_H = img.shape[0]
  img_W = img.shape[1]
  filename_txt = os.path.join(label_path, filename_jpg.replace("jpg", "txt"))
  try:
    if filename_txt.endswith(".txt"):
      with open(os.path.join(label_path, filename_txt), 'r') as f:
        lines = f.readlines()
        for line in lines:
          x_center, y_center, width, height, confid = [float(bbox) for bbox in line.split()[1:]]
          x_min = (x_center - width/2) * img_W
          y_min = (y_center - height/2) * img_H
          x_max = (x_center + width/2) * img_W
          y_max = (y_center + height/2) * img_H
          pascal_voc += (f"vehicle {x_min:.2f} {y_max:.2f} {x_max:.2f} {y_min:.2f} {confid:.2f}\n")
          print(pascal_voc)
          print(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")))
      with open(os.path.join(dest_path, filename_jpg.replace("jpg", "txt")), 'w') as f:
        f.write(pascal_voc)
  except:
    pass

vehicle 157.00 513.00 293.00 453.00 0.58

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results/023.txt
vehicle 157.00 513.00 293.00 453.00 0.58
vehicle 219.00 238.00 348.00 121.00 0.70

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results/023.txt
vehicle 157.00 513.00 293.00 453.00 0.58
vehicle 219.00 238.00 348.00 121.00 0.70
vehicle 157.00 515.00 353.00 450.00 0.71

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results/023.txt
vehicle 157.00 513.00 293.00 453.00 0.58
vehicle 219.00 238.00 348.00 121.00 0.70
vehicle 157.00 515.00 353.00 450.00 0.71
vehicle 237.00 366.00 324.00 316.00 0.80

/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/test results/023.txt
vehicle 157.00 513.00 293.00 453.00 0.58
vehicle 219.00 238.00 348.00 121.00 0.70
vehicle 157.00 515.00 353.00 450.00 0.71
vehicle 237.00 366.00 324.00 316.00 0.80
vehicle 673.00 288.00 739.00 190.00 0.87

/content/dri

#### Cycle GAN pipelines for dehaze

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition"
#!git clone "https://github.com/aitorzip/PyTorch-CycleGAN.git"
!pip3 install visdom -q

In [13]:
# prepare image for CycleGAN, split image top and bottom half from ground truth
import cv2

img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/original dataset/train/haze_images"
hazy_img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN/datasets/Hazy/train/A"
clear_img_path = "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN/datasets/Hazy/train/B"

for filename_jpg in os.listdir(img_path):
  img = cv2.imread(os.path.join(img_path, filename_jpg))
  height, width = img.shape[:2]

  # Split the image into two halves vertically
  top_half = img[0:int(height/2), 0:width]
  bottom_half = img[int(height/2):height, 0:width]

  # Save each half as a separate image file
  cv2.imwrite(os.path.join(hazy_img_path, filename_jpg), top_half)
  cv2.imwrite(os.path.join(clear_img_path, filename_jpg), bottom_half)

In [16]:
!ls -al "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN/datasets/Hazy/train/A" | wc -l
!ls -la "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN/datasets/Hazy/train/B" | wc -l

178
178


In [ ]:
# run cycle GAN

%cd "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN"
!chmod 755 *
!./train  --dataroot datasets/Hazy/ --cuda 

In [ ]:
# inference cycle GAN
%cd "/content/drive/MyDrive/Colab Notebooks/CVPR23 Object in Hazy Condition/CycleGAN"
!chmod 755 *
!./test --dataroot datasets/Hazy/test/ --cuda